In [ ]:
import pickle
import shapely
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxa
import matplotlib.pyplot as plt

# Add main repo to path
import sys
from os.path import expanduser
sys.path.append(expanduser('../../'))

from spicy_snow import retrieve_snow_depth

from spicy_snow.IO.user_dates import get_input_dates
from spicy_snow.processing.s1_preprocessing import merge_partial_s1_images
from spicy_snow.download.sentinel1 import combine_s1_images

In [ ]:
for fp in Path('/Users/zachkeskinen/Documents/spicy-snow/scripts/optimize/rmse_insitu').glob('*no_flag.nc'):
    fp_lidar = next(Path('/Users/zachkeskinen/Documents/spicy-snow/scripts/optimize/rmse_out').glob(fp.name))
    da = xr.open_dataarray(fp)
    da2 = xr.open_dataarray(fp_lidar) * 100
    insitu_best = da.where(da==da.min(), drop=True).squeeze()
    lidar_best = da2.where(da2==da2.min(), drop=True).squeeze()

    fig, axes = plt.subplots(3, 2, figsize = (12, 8))
    plt.suptitle(f'{fp.stem} RMSE Comparison')

    for i, param in enumerate(['A', 'B', 'C']):
        vmin, vmax = da.reduce(np.min, dim = [param]).quantile([.2, .8])
        vmin2, vmax2 = da2.reduce(np.min, dim = [param]).quantile([.2, .8])
        vmin = np.min([vmin, vmin2])
        vmax = np.max([vmax, vmax2])
        da.reduce(np.min, dim = [param]).plot(ax = axes[i, 0]) # vmin = vmin, vmax = vmax
        da2.reduce(np.min, dim = [param]).plot(ax = axes[i, 1]) # vmin = vmin, vmax = vmax
        axes[i, 0].set_title(f'In-Situ {param} @ {np.round(insitu_best[param].data, 2)}', fontweight='bold')
        axes[i, 1].set_title(f'Lidar {param} @ {np.round(lidar_best[param].data, 2)}', fontweight='bold')
    plt.tight_layout()
    plt.show()